## 7. Interactive Exploration

Try modifying the parameters to see how the DKW controller behaves:

### 🔧 Experiment Ideas:
1. **Adjust controller parameters**: Change `epsilon_target`, `delta`, `min_samples`, or `hysteresis`
2. **Modify the dataset**: Add more examples or change difficulty values
3. **Test edge cases**: What happens with very high or very low error rates?
4. **Analyze convergence**: How many samples does the controller need to stabilize?

### 📚 Key Insights:
- The **DKW bound** provides statistical guarantees about error rates
- **Hysteresis** prevents decision oscillation  
- The controller adapts to **empirical error patterns**
- **Fusion mode** enables efficiency gains when error rates are acceptable

### 🚀 Next Steps:
This notebook is fully self-contained and ready for experimentation. Modify any cell above and re-run to explore different scenarios!

In [ ]:
import matplotlib.pyplot as plt

# Create visualization of DKW controller behavior
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10))

# Plot 1: Decisions over time
decisions_baseline = [1 if d == 'fission' else 0 for d in baseline_df['decision']]
decisions_proposed = [1 if d == 'fission' else 0 for d in proposed_df['decision']]

ax1.step(range(len(decisions_baseline)), decisions_baseline, label='Baseline (Always Fission)', linewidth=2, alpha=0.7)
ax1.step(range(len(decisions_proposed)), decisions_proposed, label='DKW Controller', linewidth=2)
ax1.set_ylabel('Decision\n(1=Fission, 0=Fusion)')
ax1.set_title('Decision Pattern Comparison')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Error occurrence and difficulty
ax2.bar(range(len(proposed_df)), proposed_df['difficulty'], alpha=0.6, label='Difficulty', color='orange')
error_positions = [i for i, err in enumerate(proposed_df['error']) if err]
ax2.scatter(error_positions, [proposed_df.iloc[i]['difficulty'] for i in error_positions], 
           color='red', s=100, label='Actual Errors', zorder=5)
ax2.set_ylabel('Difficulty / Error Rate')
ax2.set_title('Difficulty vs Actual Errors')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Cumulative error rate
cumulative_errors_baseline = np.cumsum(baseline_df['error']) / (np.arange(len(baseline_df)) + 1)
cumulative_errors_proposed = np.cumsum(proposed_df['error']) / (np.arange(len(proposed_df)) + 1)

ax3.plot(cumulative_errors_baseline, label='Baseline Error Rate', linewidth=2, alpha=0.7)
ax3.plot(cumulative_errors_proposed, label='DKW Controller Error Rate', linewidth=2)
ax3.axhline(y=0.10, color='red', linestyle='--', alpha=0.7, label='Target Threshold (ε=0.10)')
ax3.set_xlabel('Example Index')
ax3.set_ylabel('Cumulative Error Rate')
ax3.set_title('Error Rate Evolution')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n📈 PERFORMANCE SUMMARY:")
print(f"Baseline cumulative error rate: {cumulative_errors_baseline[-1]:.3f}")
print(f"DKW Controller cumulative error rate: {cumulative_errors_proposed[-1]:.3f}")
print(f"Fusion decisions made by DKW Controller: {(proposed_df['decision'] == 'fusion').sum()}")
print(f"Efficiency gain (fusion usage): {(proposed_df['decision'] == 'fusion').sum() / len(proposed_df) * 100:.1f}%")

## 6. Visualization and Analysis

Let's visualize how the DKW controller adapts its decisions based on observed error rates.

In [ ]:
# Run the experiment
print("Running DKW Controller experiment...")
results = run_experiment(sample_data)

print("✓ Experiment completed!")
print(f"Baseline results: {len(results['baseline'])} decisions")
print(f"Proposed results: {len(results['proposed'])} decisions")

# Display results summary
baseline_df = pd.DataFrame(results['baseline'])
proposed_df = pd.DataFrame(results['proposed'])

print(f"\n📊 BASELINE (Always Fission):")
print(f"   Fission decisions: {(baseline_df['decision'] == 'fission').sum()}")
print(f"   Fusion decisions: {(baseline_df['decision'] == 'fusion').sum()}")
print(f"   Errors encountered: {baseline_df['error'].sum()}")

print(f"\n🧠 DKW CONTROLLER (Proposed):")
print(f"   Fission decisions: {(proposed_df['decision'] == 'fission').sum()}")
print(f"   Fusion decisions: {(proposed_df['decision'] == 'fusion').sum()}")  
print(f"   Errors encountered: {proposed_df['error'].sum()}")

# Show detailed comparison
print(f"\n📋 DETAILED COMPARISON:")
comparison_df = pd.DataFrame({
    'ID': proposed_df['id'],
    'Difficulty': proposed_df['difficulty'],
    'Error': proposed_df['error'],
    'Baseline': baseline_df['decision'],
    'DKW Controller': proposed_df['decision']
})
print(comparison_df)

## 5. Run the Experiment

Let's run our experiment and compare the DKW controller's decisions against the baseline approach.

In [ ]:
def run_experiment(data):
    """Run DKW controller experiment with inline data."""
    controller = DKWController()
    results = {"baseline": [], "proposed": []}

    for example in data:
        # Simulate error occurrence based on difficulty
        np.random.seed(hash(example["id"]) % 2**31)  # Deterministic randomness for reproducibility
        error = np.random.random() < example["difficulty"]
        controller.add_observation(float(error))
        decision = controller.decide()

        results["proposed"].append({
            "id": example["id"],
            "decision": decision,
            "error": error,
            "difficulty": example["difficulty"],
        })
        results["baseline"].append({
            "id": example["id"],
            "decision": "fission",  # Always conservative
            "error": error,
            "difficulty": example["difficulty"],
        })

    return results

print("✓ Experiment function defined successfully!")

## 4. Experiment Function

The `run_experiment` function simulates running our DKW controller on the test data, comparing it against a baseline that always chooses the conservative "fission" mode.

In [ ]:
# Sample dataset - inline data (no external file dependencies)
sample_data = [
    {"id": "example_000", "difficulty": 0.02},  # Low difficulty
    {"id": "example_001", "difficulty": 0.08},  # Medium difficulty  
    {"id": "example_002", "difficulty": 0.15},  # High difficulty
    {"id": "example_003", "difficulty": 0.03},  # Low difficulty
    {"id": "example_004", "difficulty": 0.12},  # Medium-high difficulty
    {"id": "example_005", "difficulty": 0.05},  # Low difficulty
    {"id": "example_006", "difficulty": 0.18},  # High difficulty
    {"id": "example_007", "difficulty": 0.01},  # Very low difficulty
    {"id": "example_008", "difficulty": 0.09},  # Medium difficulty
    {"id": "example_009", "difficulty": 0.20},  # Very high difficulty
]

print(f"✓ Sample data loaded: {len(sample_data)} examples")
print(f"Difficulty range: {min(ex['difficulty'] for ex in sample_data):.2f} - {max(ex['difficulty'] for ex in sample_data):.2f}")

# Display first few examples
import pandas as pd
df = pd.DataFrame(sample_data[:5])
print(f"\nFirst 5 examples:")
print(df)

## 3. Sample Data (Self-Contained)

Instead of reading from external JSON files, we'll inline the test data directly in the notebook. This includes sample examples with varying difficulty levels that simulate real-world scenarios.

In [ ]:
@dataclass
class DKWController:
    """DKW-guided fusion/fission controller."""
    epsilon_target: float = 0.10
    delta: float = 0.05
    min_samples: int = 100
    hysteresis: float = 0.05

    samples: list = field(default_factory=list)
    current_state: str = "fission"

    def dkw_epsilon(self, n: int) -> float:
        """Compute DKW epsilon for n samples."""
        if n < 2:
            return 1.0
        return np.sqrt(np.log(2 / self.delta) / (2 * n))

    def add_observation(self, error: float) -> None:
        """Add error observation for calibration."""
        self.samples.append(error)

    def decide(self) -> str:
        """Make fusion/fission decision with DKW guarantee."""
        n = len(self.samples)
        if n < self.min_samples:
            return self.current_state

        epsilon = self.dkw_epsilon(n)
        empirical_error = np.mean(self.samples[-self.min_samples:])
        error_upper_bound = empirical_error + epsilon

        if self.current_state == "fusion":
            if error_upper_bound > self.epsilon_target + self.hysteresis:
                self.current_state = "fission"
        else:
            if error_upper_bound < self.epsilon_target - self.hysteresis:
                self.current_state = "fusion"

        return self.current_state

# Test the controller
controller = DKWController()
print("✓ DKWController class created successfully!")
print(f"Initial state: {controller.current_state}")
print(f"Target epsilon: {controller.epsilon_target}")
print(f"Min samples required: {controller.min_samples}")

## 2. DKW Controller Class

The `DKWController` uses the **Dvoretzky-Kiefer-Wolfowitz inequality** to provide statistical confidence bounds on empirical error rates.

### Key Parameters:
- `epsilon_target`: Target error threshold (default: 0.10)
- `delta`: Confidence parameter for DKW bound (default: 0.05)  
- `min_samples`: Minimum samples before making decisions (default: 100)
- `hysteresis`: Prevents state oscillation (default: 0.05)

### States:
- **Fusion**: Aggressive mode (lower latency, higher risk)
- **Fission**: Conservative mode (higher latency, lower risk)

In [ ]:
"""DKW Controller Implementation."""
import json
import numpy as np
from dataclasses import dataclass, field

print("✓ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")

# DKW Controller Implementation - Interactive Demo

This notebook implements a **DKW-guided fusion/fission controller** that makes decisions based on error observations with statistical guarantees.

## Overview
- **DKW (Dvoretzky-Kiefer-Wolfowitz)** inequality provides confidence bounds for empirical error rates
- **Fusion/Fission** decisions control system behavior based on error thresholds
- **Hysteresis** prevents oscillation between states
- **Self-contained demo** with inline data - no external files required

## 1. Imports and Setup

Let's start by importing the necessary libraries for our DKW controller implementation.